## OVERVIEW

This notebook deals with the following data: \
previous_application \
POS_CASH_balance \
installments_payments \
credit_card_balance \
Additional features for the application data set. Not implemented, just included. The goal is to join all these dataframes together using the SK_ID_CURR column with the balance dataset as the base. Feel Free to add any new features you find. It's easiest to make them by themselves and then just join them on at the end.

#### Packages

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
path = '/content/drive/MyDrive/project/home-credit-default-risk/'

#### One hot Encoder Function

In [3]:
def encoder(df):
    col_list = []
    
    for col in df.columns:
        if df[col].dtype =="object":
            col_list.append(col)
    
    df_new = pd.get_dummies(df, columns=col_list)
    return df_new

#### Previous Application

In [4]:
prev_app = pd.read_csv(path+'previous_application.csv')

In [5]:
#PREV APPLICATION prev_PRODUCT_COMBINATION: PRODUCT_COMBINATION value from most recent previous application.

prev_app_latest = prev_app[['SK_ID_PREV','SK_ID_CURR']] \
    .groupby('SK_ID_CURR').agg({'SK_ID_PREV': 'first'})
prev_app_comb = prev_app.loc[prev_app['SK_ID_PREV'] \
                             .isin(prev_app_latest['SK_ID_PREV'])]


prev_app_comb = prev_app_comb[['SK_ID_CURR','SK_ID_PREV','PRODUCT_COMBINATION']]

prev_app_comb.set_index('SK_ID_CURR',inplace=True)

prev_app_comb.drop(columns = ['SK_ID_PREV'], inplace=True)
prev_app_comb.columns = ['MOST_RECENT_PRODUCT_COMBINATION']
prev_app_comb = encoder(prev_app_comb)

In [6]:
#One Hot Encoding previous Applications
prev_app = encoder(prev_app)

In [7]:
#Cleaning Some bad data
prev_app['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

In [8]:
#Adding non agg variable
prev_app['APP_CREDIT_PERC'] = np.where(prev_app['AMT_CREDIT'] != 0,
                                       prev_app['AMT_APPLICATION'] / prev_app['AMT_CREDIT'],
                                       np.nan)

In [9]:
prev_app.drop(['SK_ID_PREV'], axis= 1, inplace = True)

In [10]:
#Dictionary of various aggregations to be performed. If column not shown, assume only mean was performed
num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }

for col in prev_app.columns:
    if col not in num_aggregations.keys() and col != 'SK_ID_CURR':
        num_aggregations[col] = ['mean']

In [11]:
prev_app_agg = prev_app.groupby('SK_ID_CURR').agg(num_aggregations)

prev_app_agg.columns = ['_'.join(col) for col in prev_app_agg.columns.values]

In [12]:
#Splitting into approved vs refused

approved = prev_app[prev_app['NAME_CONTRACT_STATUS_Approved'] == 1]

approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)

approved_agg.columns = ['_'.join(col) for col in approved_agg.columns.values]

approved_agg = approved_agg.add_suffix('_Approved')

In [13]:
#Refused
refused = prev_app[prev_app['NAME_CONTRACT_STATUS_Refused'] == 1]

refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)

refused_agg.columns = ['_'.join(col) for col in refused_agg.columns.values]

refused_agg = refused_agg.add_suffix('_Refused')

# status_agg = approved_agg.append(refused_agg).reset_index()

In [14]:
#Joining back on to the main agg df. _Status suffix indicates which columns this was performed on
prev_app_agg = prev_app_agg.join([approved_agg, refused_agg])

In [15]:
prev_app_agg = prev_app_agg.join(prev_app_comb, how='left')

In [16]:
#Removing created df's
del prev_app_latest, prev_app_comb, approved, approved_agg, refused, refused_agg

In [17]:
prev_app_agg

,AMT_ANNUITY_min,AMT_ANNUITY_max,AMT_ANNUITY_mean,AMT_APPLICATION_min,AMT_APPLICATION_max,AMT_APPLICATION_mean,AMT_CREDIT_min,AMT_CREDIT_max,AMT_CREDIT_mean,APP_CREDIT_PERC_min,APP_CREDIT_PERC_max,APP_CREDIT_PERC_mean,APP_CREDIT_PERC_var,AMT_DOWN_PAYMENT_min,AMT_DOWN_PAYMENT_max,AMT_DOWN_PAYMENT_mean,AMT_GOODS_PRICE_min,AMT_GOODS_PRICE_max,AMT_GOODS_PRICE_mean,HOUR_APPR_PROCESS_START_min,HOUR_APPR_PROCESS_START_max,HOUR_APPR_PROCESS_START_mean,RATE_DOWN_PAYMENT_min,RATE_DOWN_PAYMENT_max,RATE_DOWN_PAYMENT_mean,DAYS_DECISION_min,DAYS_DECISION_max,DAYS_DECISION_mean,CNT_PAYMENT_mean,CNT_PAYMENT_sum,NFLAG_LAST_APPL_IN_DAY_mean,RATE_INTEREST_PRIMARY_mean,RATE_INTEREST_PRIVILEGED_mean,SELLERPLACE_AREA_mean,DAYS_FIRST_DRAWING_mean,DAYS_FIRST_DUE_mean,DAYS_LAST_DUE_1ST_VERSION_mean,DAYS_LAST_DUE_mean,DAYS_TERMINATION_mean,NFLAG_INSURED_ON_APPROVAL_mean,...,NAME_SELLER_INDUSTRY_XNA_mean_Refused,NAME_YIELD_GROUP_XNA_mean_Refused,NAME_YIELD_GROUP_high_mean_Refused,NAME_YIELD_GROUP_low_action_mean_Refused,NAME_YIELD_GROUP_low_normal_mean_Refused,NAME_YIELD_GROUP_middle_mean_Refused,PRODUCT_COMBINATION_Card Street_mean_Refused,PRODUCT_COMBINATION_Card X-Sell_mean_Refused,PRODUCT_COMBINATION_Cash_mean_Refused,PRODUCT_COMBINATION_Cash Street: high_mean_Refused,PRODUCT_COMBINATION_Cash Street: low_mean_Refused,PRODUCT_COMBINATION_Cash Street: middle_mean_Refused,PRODUCT_COMBINATION_Cash X-Sell: high_mean_Refused,PRODUCT_COMBINATION_Cash X-Sell: low_mean_Refused,PRODUCT_COMBINATION_Cash X-Sell: middle_mean_Refused,PRODUCT_COMBINATION_POS household with interest_mean_Refused,PRODUCT_COMBINATION_POS household without interest_mean_Refused,PRODUCT_COMBINATION_POS industry with interest_mean_Refused,PRODUCT_COMBINATION_POS industry without interest_mean_Refused,PRODUCT_COMBINATION_POS mobile with interest_mean_Refused,PRODUCT_COMBINATION_POS mobile without interest_mean_Refused,PRODUCT_COMBINATION_POS other with interest_mean_Refused,PRODUCT_COMBINATION_POS others without interest_mean_Refused,MOST_RECENT_PRODUCT_COMBINATION_Card Street,MOST_RECENT_PRODUCT_COMBINATION_Card X-Sell,MOST_RECENT_PRODUCT_COMBINATION_Cash,MOST_RECENT_PRODUCT_COMBINATION_Cash Street: high,MOST_RECENT_PRODUCT_COMBINATION_Cash Street: low,MOST_RECENT_PRODUCT_COMBINATION_Cash Street: middle,MOST_RECENT_PRODUCT_COMBINATION_Cash X-Sell: high,MOST_RECENT_PRODUCT_COMBINATION_Cash X-Sell: low,MOST_RECENT_PRODUCT_COMBINATION_Cash X-Sell: middle,MOST_RECENT_PRODUCT_COMBINATION_POS household with interest,MOST_RECENT_PRODUCT_COMBINATION_POS household without interest,MOST_RECENT_PRODUCT_COMBINATION_POS industry with interest,MOST_RECENT_PRODUCT_COMBINATION_POS industry without interest,MOST_RECENT_PRODUCT_COMBINATION_POS mobile with interest,MOST_RECENT_PRODUCT_COMBINATION_POS mobile without interest,MOST_RECENT_PRODUCT_COMBINATION_POS other with interest,MOST_RECENT_PRODUCT_COMBINATION_POS others without interest
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000000,24835.5,24835.5,24835.500,23787.0,23787.0,23787.00,1.044079,1.044079,1.044079,NaN,2520.0,2520.0,2520.00,24835.5,24835.5,24835.500,13,13,13.000000,0.104326,0.104326,0.104326,-1740,-1740,-1740.000,8.00,8.0,1.0,NaN,NaN,23.000,NaN,-1709.000000,-1499.000000,-1619.000000,-1612.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
100002,9251.775,9251.775,9251.775000,179055.0,179055.0,179055.000,179055.0,179055.0,179055.00,1.000000,1.000000,1.000000,NaN,0.0,0.0,0.00,179055.0,179055.0,179055.000,9,9,9.000000,0.000000,0.000000,0.000000,-606,-606,-606.000,24.00,24.0,1.0,NaN,NaN,500.000,NaN,-565.000000,125.000000,-25.000000,-17.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
100003,6737.310,98356.995,56553.990000,68809.5,900000.0,435436.500,68053.5,1035882.0,484191.00,0

#### POS CASH

In [18]:
pos = pd.read_csv(path+'POS_CASH_balance.csv')

In [19]:
pos = encoder(pos)

In [20]:
pos.drop(['SK_ID_PREV'], axis= 1, inplace = True)

In [21]:
aggregations = {
    'MONTHS_BALANCE': ['max', 'mean', 'size'],
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean'],
    'CNT_INSTALMENT': ['mean'],
    'CNT_INSTALMENT_FUTURE': ['mean']
    }

for col in pos.columns:
    if col not in aggregations.keys() and col != 'SK_ID_CURR':
        aggregations[col] = ['sum']

In [22]:
pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)

pos_agg.columns = ['_'.join(col) for col in pos_agg.columns.values]

# Count pos cash accounts
pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()

In [23]:
del pos

In [24]:
pos_agg

,MONTHS_BALANCE_max,MONTHS_BALANCE_mean,MONTHS_BALANCE_size,SK_DPD_max,SK_DPD_mean,SK_DPD_DEF_max,SK_DPD_DEF_mean,CNT_INSTALMENT_mean,CNT_INSTALMENT_FUTURE_mean,NAME_CONTRACT_STATUS_Active_sum,NAME_CONTRACT_STATUS_Amortized debt_sum,NAME_CONTRACT_STATUS_Approved_sum,NAME_CONTRACT_STATUS_Canceled_sum,NAME_CONTRACT_STATUS_Completed_sum,NAME_CONTRACT_STATUS_Demand_sum,NAME_CONTRACT_STATUS_Returned to the store_sum,NAME_CONTRACT_STATUS_Signed_sum,NAME_CONTRACT_STATUS_XNA_sum,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,
100001,-53,-72.555556,9,7,0.777778,7,0.777778,4.000000,1.444444,7.0,0,0,0,2,0,0,0,0,9
100002,-1,-10.000000,19,0,0.000000,0,0.000000,24.000000,15.000000,19.0,0,0,0,0,0,0,0,0,19
100003,-18,-43.785714,28,0,0.000000,0,0.000000,10.107143,5.785714,26.0,0,0,0,2,0,0,0,0,28
100004,-24,-25.500000,4,0,0.000000,0,0.000000,3.750000,2.250000,3.0,0,0,0,1,0,0,0,0,4
100005,-15,-20.000000,11,0,0.000000,0,0.000000,11.700000,7.200000,9.0,0,0,0,1,0,0,1,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,-1,-5.000000,9,0,0.000000,0,0.000000,7.875000,4.375000,7.0,0,0,0,1,0,0,1,0,9
456252,-76,-79.000000,7,0,0.000000,0,0.000000,6.000000,3.000000,6.0,0,0,0,1,0,0,0,0,7
456253,-57,-79.235294,17,5,0.294118,5,0.294118,6.705882,2.000000,15.0,0,0,0,2,0,0,0,0,17


#### Installments Payments

In [25]:
insta = pd.read_csv(path+'installments_payments.csv')

In [26]:
insta = encoder(insta)

In [27]:
# Percentage and difference paid in each installment (amount paid and installment value)
insta['PAYMENT_PERC'] = np.where(insta['AMT_INSTALMENT'] != 0,
                                 insta['AMT_PAYMENT'] / insta['AMT_INSTALMENT'],
                                 np.nan)
insta['PAYMENT_DIFF'] = insta['AMT_INSTALMENT'] - insta['AMT_PAYMENT']

#This one showed as a useful feature, however with the both ID's in the group by it has more rows than the standard df
insta_p = insta[insta['DAYS_INSTALMENT'] >= -1000].groupby('SK_ID_CURR').agg({'PAYMENT_DIFF':'sum'})

In [28]:
# Days past due and days before due (no negative values)
insta['DPD'] = insta['DAYS_ENTRY_PAYMENT'] - insta['DAYS_INSTALMENT']
insta['DBD'] = insta['DAYS_INSTALMENT'] - insta['DAYS_ENTRY_PAYMENT']
insta['DPD'] = insta['DPD'].where(insta['DPD'] > 0, 0)
insta['DBD'] = insta['DBD'].where(insta['DBD'] > 0, 0)

In [29]:
#Not sure if need this col or not in order to merge insta_p
insta.drop(['SK_ID_PREV'], axis= 1, inplace = True)

In [30]:
aggregations = {
    'NUM_INSTALMENT_VERSION': ['nunique'],
    'DPD': ['max', 'mean', 'sum'],
    'DBD': ['max', 'mean', 'sum'],
    'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
    'AMT_INSTALMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }

for col in insta.columns:
    if col not in aggregations.keys() and col != 'SK_ID_CURR':
        aggregations[col] = ['mean']

In [31]:
insta_agg = insta.groupby('SK_ID_CURR').agg(aggregations)

insta_agg.columns = ['_'.join(col) for col in insta_agg.columns.values]

In [32]:
# Count installments accounts
insta_agg['INSTAL_COUNT'] = insta.groupby('SK_ID_CURR').size()

In [33]:
insta_agg = insta_agg.join(insta_p, how='left', on='SK_ID_CURR')

In [34]:
del insta, insta_p

In [35]:
insta_agg

,NUM_INSTALMENT_VERSION_nunique,DPD_max,DPD_mean,DPD_sum,DBD_max,DBD_mean,DBD_sum,PAYMENT_PERC_max,PAYMENT_PERC_mean,PAYMENT_PERC_sum,PAYMENT_PERC_var,PAYMENT_DIFF_max,PAYMENT_DIFF_mean,PAYMENT_DIFF_sum,PAYMENT_DIFF_var,AMT_INSTALMENT_max,AMT_INSTALMENT_mean,AMT_INSTALMENT_sum,AMT_PAYMENT_min,AMT_PAYMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_sum,DAYS_ENTRY_PAYMENT_max,DAYS_ENTRY_PAYMENT_mean,DAYS_ENTRY_PAYMENT_sum,NUM_INSTALMENT_NUMBER_mean,DAYS_INSTALMENT_mean,INSTAL_COUNT,PAYMENT_DIFF
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,2,11.0,1.571429,11.0,36.0,8.857143,62.0,1.00000,1.000000,7.000000,0.000000,0.000,0.00000,0.000,0.000000e+00,17397.900,5885.132143,41195.925,3951.000,17397.900,5885.132143,41195.925,-1628.0,-2195.000000,-15365.0,2.714286,-2187.714286,7,NaN
100002,2,0.0,0.000000,0.0,31.0,20.421053,388.0,1.00000,1.000000,19.000000,0.000000,0.000,0.00000,0.000,0.000000e+00,53093.745,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,-49.0,-315.421053,-5993.0,10.000000,-295.000000,19,0.000
100003,2,0.0,0.000000,0.0,14.0,7.160000,179.0,1.00000,1.000000,25.000000,0.000000,0.000,0.00000,0.000,0.000000e+00,560835.360,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,-544.0,-1385.320000,-34633.0,5.080000,-1378.160000,25,0.000
100004,2,0.0,0.000000,0.0,11.0,7.666667,23.0,1.00000,1.000000,3.000000,0.000000,0.000,0.00000,0.000,0.000000e+00,10573.965,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,-727.0,-761.666667,-2285.0,2.000000,-754.000000,3,0.000
100005,2,1.0,0.111111,1.0,37.0,23.666667,213.0,1.00000,1.000000,9.000000,0.000000,0.000,0.00000,0.000,0.000000e+00,17656.245,6240.205000,56161.845,4813.200,17656.245,6240.205000,56161.845,-470.0,-609.555556,-5486.0,5.000000,-586.000000,9,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,2,0.0,0.000000,0.0,46.0,36.285714,254.0,1.00000,1.000000,7.000000,0.000000,0.000,0.00000,0.000,0.000000e+00,12815.010,7492.924286,52450.470,6605.910,12815.010,7492.924286,52450.470,-38.0,-156.285714,-1094.0,4.000000,-120.000000,7,0.000
456252,1,3.0,0.500000,3.0,11.0,3.333333,20.0,1.00000,1.000000,6.000000,0.000000,0.000,0.00000,0.000,0.000000e+00,10074.465,10069.867500,60419.205,10046.880,10074.465,10069.867500,60419.205,-2327.0,-2393.833333,-14363.0,3.500000,-2391.000000,6,NaN
456253,1,9.0,0.642857,9.0,51.0,15.142857,212.0,1.00000,0.928571,13.000000,0.070380,3945.825,283.79250,3973.095,1.110980e+06,5575.185,4399.707857,61595.910,27.270,5575.185,4115.915357,57622.815,-1738.0,-2387.428571,-33424.0,4.785714,-2372.928571,14,NaN


#### Credit Card Balance

In [36]:
credit = pd.read_csv(path+'credit_card_balance.csv')

In [37]:
credit = encoder(credit)

In [38]:
credit.drop(['SK_ID_PREV'], axis= 1, inplace = True)

In [39]:
credit_agg = credit.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])

credit_agg.columns = ['_'.join(col) +'_credit' for col in credit_agg.columns.values]

In [40]:
credit_agg['CC_COUNT'] = credit.groupby('SK_ID_CURR').size()

In [41]:
del credit

In [42]:
credit_agg

,MONTHS_BALANCE_min_credit,MONTHS_BALANCE_max_credit,MONTHS_BALANCE_mean_credit,MONTHS_BALANCE_sum_credit,MONTHS_BALANCE_var_credit,AMT_BALANCE_min_credit,AMT_BALANCE_max_credit,AMT_BALANCE_mean_credit,AMT_BALANCE_sum_credit,AMT_BALANCE_var_credit,AMT_CREDIT_LIMIT_ACTUAL_min_credit,AMT_CREDIT_LIMIT_ACTUAL_max_credit,AMT_CREDIT_LIMIT_ACTUAL_mean_credit,AMT_CREDIT_LIMIT_ACTUAL_sum_credit,AMT_CREDIT_LIMIT_ACTUAL_var_credit,AMT_DRAWINGS_ATM_CURRENT_min_credit,AMT_DRAWINGS_ATM_CURRENT_max_credit,AMT_DRAWINGS_ATM_CURRENT_mean_credit,AMT_DRAWINGS_ATM_CURRENT_sum_credit,AMT_DRAWINGS_ATM_CURRENT_var_credit,AMT_DRAWINGS_CURRENT_min_credit,AMT_DRAWINGS_CURRENT_max_credit,AMT_DRAWINGS_CURRENT_mean_credit,AMT_DRAWINGS_CURRENT_sum_credit,AMT_DRAWINGS_CURRENT_var_credit,AMT_DRAWINGS_OTHER_CURRENT_min_credit,AMT_DRAWINGS_OTHER_CURRENT_max_credit,AMT_DRAWINGS_OTHER_CURRENT_mean_credit,AMT_DRAWINGS_OTHER_CURRENT_sum_credit,AMT_DRAWINGS_OTHER_CURRENT_var_credit,AMT_DRAWINGS_POS_CURRENT_min_credit,AMT_DRAWINGS_POS_CURRENT_max_credit,AMT_DRAWINGS_POS_CURRENT_mean_credit,AMT_DRAWINGS_POS_CURRENT_sum_credit,AMT_DRAWINGS_POS_CURRENT_var_credit,AMT_INST_MIN_REGULARITY_min_credit,AMT_INST_MIN_REGULARITY_max_credit,AMT_INST_MIN_REGULARITY_mean_credit,AMT_INST_MIN_REGULARITY_sum_credit,AMT_INST_MIN_REGULARITY_var_credit,...,SK_DPD_DEF_max_credit,SK_DPD_DEF_mean_credit,SK_DPD_DEF_sum_credit,SK_DPD_DEF_var_credit,NAME_CONTRACT_STATUS_Active_min_credit,NAME_CONTRACT_STATUS_Active_max_credit,NAME_CONTRACT_STATUS_Active_mean_credit,NAME_CONTRACT_STATUS_Active_sum_credit,NAME_CONTRACT_STATUS_Active_var_credit,NAME_CONTRACT_STATUS_Approved_min_credit,NAME_CONTRACT_STATUS_Approved_max_credit,NAME_CONTRACT_STATUS_Approved_mean_credit,NAME_CONTRACT_STATUS_Approved_sum_credit,NAME_CONTRACT_STATUS_Approved_var_credit,NAME_CONTRACT_STATUS_Completed_min_credit,NAME_CONTRACT_STATUS_Completed_max_credit,NAME_CONTRACT_STATUS_Completed_mean_credit,NAME_CONTRACT_STATUS_Completed_sum_credit,NAME_CONTRACT_STATUS_Completed_var_credit,NAME_CONTRACT_STATUS_Demand_min_credit,NAME_CONTRACT_STATUS_Demand_max_credit,NAME_CONTRACT_STATUS_Demand_mean_credit,NAME_CONTRACT_STATUS_Demand_sum_credit,NAME_CONTRACT_STATUS_Demand_var_credit,NAME_CONTRACT_STATUS_Refused_min_credit,NAME_CONTRACT_STATUS_Refused_max_credit,NAME_CONTRACT_STATUS_Refused_mean_credit,NAME_CONTRACT_STATUS_Refused_sum_credit,NAME_CONTRACT_STATUS_Refused_var_credit,NAME_CONTRACT_STATUS_Sent proposal_min_credit,NAME_CONTRACT_STATUS_Sent proposal_max_credit,NAME_CONTRACT_STATUS_Sent proposal_mean_credit,NAME_CONTRACT_STATUS_Sent proposal_sum_credit,NAME_CONTRACT_STATUS_Sent proposal_var_credit,NAME_CONTRACT_STATUS_Signed_min_credit,NAME_CONTRACT_STATUS_Signed_max_credit,NAME_CONTRACT_STATUS_Signed_mean_credit,NAME_CONTRACT_STATUS_Signed_sum_credit,NAME_CONTRACT_STATUS_Signed_var_credit,CC_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100006,-6,-1,-3.5,-21,3.5,0.000,0.000,0.000000,0.000,0.000000e+00,270000,270000,270000.000000,1620000,0.000000e+00,NaN,NaN,NaN,0.0,NaN,0.0,0.00,0.000000,0.000,0.000000e+00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.000,NaN,0.0,0.000,0.000000,0.000,0.000000e+00,...,0,0.000000,0,0.000000,1,1,1.000000,6,0.000000,0,0,0.0,0,0.0,0,0,0.000000,0,0.000000,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,6
100011,-75,-2,-38.5,-2849,462.5,0.000,189000.000,54482.111149,4031676.225,4.641321e+09,90000,180000,164189.189189,12150000,1.189060e+09,0.0,180000.0,2432.432432,180000.0,4.378378e+08,0.0,180000.00,2432.432432,180000.000,4.378378e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.000,0.000000e+00,0.0,9000.000,3956.221849,288804.195,2.013991e+07,...,0,0.000000,0,0.000000,1,1,1.000000,74,0.000000,0,0,0.0,0,0.0,0,0,0.000000,0,0.000000,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,74
100013,-96,-1,-48.5,-4656,776.0,0.000,161420.220,18159.919219,1743352.245,1.869473e+09,45000,157500,131718.750000,12645000,2.259252e+09,0.0,157500.0,6350.000000,571500.0

#### Bureau Table (ignoring Bureau Balance for now)

In [43]:
bur = pd.read_csv(path+'bureau.csv')

In [44]:
sum_col = bur[['SK_ID_CURR','CREDIT_DAY_OVERDUE','DAYS_CREDIT_ENDDATE','AMT_CREDIT_MAX_OVERDUE',
             'AMT_CREDIT_SUM','AMT_CREDIT_SUM_DEBT','AMT_CREDIT_SUM_LIMIT',
              'AMT_CREDIT_SUM_OVERDUE','CREDIT_TYPE','DAYS_CREDIT_UPDATE',
             'AMT_ANNUITY']].groupby('SK_ID_CURR').sum()
sum_col.rename(columns={'AMT_ANNUITY': 'BUREAU_AMT_ANNUITY'}, inplace=True)

In [45]:
min_col = bur[['SK_ID_CURR','DAYS_CREDIT']].groupby('SK_ID_CURR').min()
min_col.columns = ['DAYS_CREDIT_OLDEST_ACCOUNT']

In [46]:
max_col = bur[['SK_ID_CURR','DAYS_CREDIT']].groupby('SK_ID_CURR').max()
max_col.columns = ['DAYS_CREDIT_NEWEST_ACCOUNT']

In [47]:
count_col = bur[['SK_ID_CURR','SK_ID_BUREAU']].groupby('SK_ID_CURR').count()
count_col.columns = ['NUM_CREDIT_ACCOUNTS']

In [48]:
type_col = bur[['SK_ID_CURR','CREDIT_ACTIVE','SK_ID_BUREAU']].groupby(['SK_ID_CURR','CREDIT_ACTIVE']).count()
type_col.reset_index(1, inplace=True)
type_col = type_col.pivot(columns='CREDIT_ACTIVE', values='SK_ID_BUREAU').fillna(0)
type_col.columns = [ n.upper().replace(' ','_') + '_ACCOUNTS' for n in type_col.columns.to_list()]

In [49]:
one_hot = bur[['SK_ID_CURR','CREDIT_TYPE']].copy()
one_hot['CREDIT_TYPE'] = one_hot['CREDIT_TYPE'].str.replace(' ','_').str.upper()
one_hot = pd.get_dummies(one_hot, columns=['CREDIT_TYPE'])
one_hot = one_hot.groupby('SK_ID_CURR').sum()

In [50]:
bur_final = count_col.join([sum_col, min_col, max_col, type_col, one_hot])

In [51]:
del sum_col, min_col, max_col, type_col, one_hot

In [52]:
bur_final

,NUM_CREDIT_ACCOUNTS,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,BUREAU_AMT_ANNUITY,DAYS_CREDIT_OLDEST_ACCOUNT,DAYS_CREDIT_NEWEST_ACCOUNT,ACTIVE_ACCOUNTS,BAD_DEBT_ACCOUNTS,CLOSED_ACCOUNTS,SOLD_ACCOUNTS,CREDIT_TYPE_ANOTHER_TYPE_OF_LOAN,CREDIT_TYPE_CAR_LOAN,CREDIT_TYPE_CASH_LOAN_(NON-EARMARKED),CREDIT_TYPE_CONSUMER_CREDIT,CREDIT_TYPE_CREDIT_CARD,CREDIT_TYPE_INTERBANK_CREDIT,CREDIT_TYPE_LOAN_FOR_BUSINESS_DEVELOPMENT,CREDIT_TYPE_LOAN_FOR_PURCHASE_OF_SHARES_(MARGIN_LENDING),CREDIT_TYPE_LOAN_FOR_THE_PURCHASE_OF_EQUIPMENT,CREDIT_TYPE_LOAN_FOR_WORKING_CAPITAL_REPLENISHMENT,CREDIT_TYPE_MICROLOAN,CREDIT_TYPE_MOBILE_OPERATOR_LOAN,CREDIT_TYPE_MORTGAGE,CREDIT_TYPE_REAL_ESTATE_LOAN,CREDIT_TYPE_UNKNOWN_TYPE_OF_LOAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,7,0,577.0,0.000,1453365.000,596686.500,0.000,0.0,-652,24817.500,-1572,-49,3.0,0.0,4.0,0.0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0
100002,8,0,-2094.0,8405.145,865055.565,245781.000,31988.565,0.0,-3999,0.000,-1437,-103,2.0,0.0,6.0,0.0,0,0,0,4,4,0,0,0,0,0,0,0,0,0,0
100003,4,0,-2178.0,0.000,1017400.500,0.000,810000.000,0.0,-3264,0.000,-2586,-606,1.0,0.0,3.0,0.0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0
100004,2,0,-977.0,0.000,189037.800,0.000,0.000,0.0,-1064,0.000,-1326,-408,0.0,0.0,2.0,0.0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
100005,3,0,1318.0,0.000,657126.000,568408.500,0.000,0.0,-163,4261.500,-373,-62,2.0,0.0,1.0,0.0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,13,0,-14788.0,30735.000,3693858.660,163071.000,0.000,0.0,-13839,0.000,-2713,-483,2.0,0.0,11.0,0.0,1,0,0,9,3,0,0,0,0,0,0,0,0,0,0
456250,3,0,3865.0,0.000,3086459.550,2232040.095,58268.385,0.0,-181,463703.895,-1002,-760,2.0,0.0,1.0,0.0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0
456253,4,0,1122.0,0.000,3960000.000,1795833.000,0.000,0.0,-1013,175108.500,-919,-713,2.0,0.0,2.0,0.0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0


In [53]:
bur_bal = pd.read_csv(path+'bureau_balance.csv')

In [54]:
bur_bal = encoder(bur_bal)

In [55]:
bur_bal = bur_bal.groupby('SK_ID_BUREAU').agg({'MONTHS_BALANCE': 'count',
                                               'STATUS_0': 'sum',
                                               'STATUS_1': 'sum',
                                               'STATUS_2': 'sum',
                                               'STATUS_3': 'sum',
                                               'STATUS_4': 'sum',
                                               'STATUS_5': 'sum',
                                               'STATUS_C': 'sum',
                                               'STATUS_X': 'sum'})

In [56]:
bur_bal['STATUS_0'] = bur_bal['STATUS_0'] / bur_bal['MONTHS_BALANCE']
bur_bal['STATUS_1'] = bur_bal['STATUS_1'] / bur_bal['MONTHS_BALANCE']
bur_bal['STATUS_2'] = bur_bal['STATUS_2'] / bur_bal['MONTHS_BALANCE']
bur_bal['STATUS_3'] = bur_bal['STATUS_3'] / bur_bal['MONTHS_BALANCE']
bur_bal['STATUS_4'] = bur_bal['STATUS_4'] / bur_bal['MONTHS_BALANCE']
bur_bal['STATUS_5'] = bur_bal['STATUS_5'] / bur_bal['MONTHS_BALANCE']
bur_bal['STATUS_C'] = bur_bal['STATUS_C'] / bur_bal['MONTHS_BALANCE']
bur_bal['STATUS_X'] = bur_bal['STATUS_X'] / bur_bal['MONTHS_BALANCE']

In [57]:
bur_comb = bur[['SK_ID_CURR', 'SK_ID_BUREAU']].join(bur_bal, on='SK_ID_BUREAU')

In [58]:
bur_comb = bur_comb[~bur_comb['MONTHS_BALANCE'].isna()]

In [59]:
aggregations = {}

for col in bur_comb.columns:
    if col != 'SK_ID_CURR' and col != 'SK_ID_BUREAU':
        aggregations[col] = ['mean', 'min', 'max']

In [60]:
bur_agg = bur_comb.groupby('SK_ID_CURR').agg(aggregations)
bur_agg.columns = ['_'.join(col)+'bur_bal' for col in bur_agg.columns.values]

In [61]:
bur_final = bur_final.join(bur_agg)

In [62]:
del bur, bur_agg, bur_comb, bur_bal

In [63]:
bur_final

,NUM_CREDIT_ACCOUNTS,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,BUREAU_AMT_ANNUITY,DAYS_CREDIT_OLDEST_ACCOUNT,DAYS_CREDIT_NEWEST_ACCOUNT,ACTIVE_ACCOUNTS,BAD_DEBT_ACCOUNTS,CLOSED_ACCOUNTS,SOLD_ACCOUNTS,CREDIT_TYPE_ANOTHER_TYPE_OF_LOAN,CREDIT_TYPE_CAR_LOAN,CREDIT_TYPE_CASH_LOAN_(NON-EARMARKED),CREDIT_TYPE_CONSUMER_CREDIT,CREDIT_TYPE_CREDIT_CARD,CREDIT_TYPE_INTERBANK_CREDIT,CREDIT_TYPE_LOAN_FOR_BUSINESS_DEVELOPMENT,CREDIT_TYPE_LOAN_FOR_PURCHASE_OF_SHARES_(MARGIN_LENDING),CREDIT_TYPE_LOAN_FOR_THE_PURCHASE_OF_EQUIPMENT,CREDIT_TYPE_LOAN_FOR_WORKING_CAPITAL_REPLENISHMENT,CREDIT_TYPE_MICROLOAN,CREDIT_TYPE_MOBILE_OPERATOR_LOAN,CREDIT_TYPE_MORTGAGE,CREDIT_TYPE_REAL_ESTATE_LOAN,CREDIT_TYPE_UNKNOWN_TYPE_OF_LOAN,MONTHS_BALANCE_meanbur_bal,MONTHS_BALANCE_minbur_bal,MONTHS_BALANCE_maxbur_bal,STATUS_0_meanbur_bal,STATUS_0_minbur_bal,STATUS_0_maxbur_bal,STATUS_1_meanbur_bal,STATUS_1_minbur_bal,STATUS_1_maxbur_bal,STATUS_2_meanbur_bal,STATUS_2_minbur_bal,STATUS_2_maxbur_bal,STATUS_3_meanbur_bal,STATUS_3_minbur_bal,STATUS_3_maxbur_bal,STATUS_4_meanbur_bal,STATUS_4_minbur_bal,STATUS_4_maxbur_bal,STATUS_5_meanbur_bal,STATUS_5_minbur_bal,STATUS_5_maxbur_bal,STATUS_C_meanbur_bal,STATUS_C_minbur_bal,STATUS_C_maxbur_bal,STATUS_X_meanbur_bal,STATUS_X_minbur_bal,STATUS_X_maxbur_bal
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,7,0,577.0,0.000,1453365.000,596686.500,0.000,0.0,-652,24817.500,-1572,-49,3.0,0.0,4.0,0.0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,24.571429,2.0,52.0,0.336651,0.019231,1.000000,0.007519,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.441240,0.000000,0.966667,0.214590,0.000000,0.500000
100002,8,0,-2094.0,8405.145,865055.565,245781.000,31988.565,0.0,-3999,0.000,-1437,-103,2.0,0.0,6.0,0.0,0,0,0,4,4,0,0,0,0,0,0,0,0,0,0,13.750000,4.0,22.0,0.406960,0.187500,0.818182,0.255682,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.175426,0.000000,0.812500,0.161932,0.000000,0.500000
100003,4,0,-2178.0,0.000,1017400.500,0.000,810000.000,0.0,-3264,0.000,-2586,-606,1.0,0.0,3.0,0.0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,2,0,-977.0,0.000,189037.800,0.000,0.000,0.0,-1064,0.000,-1326,-408,0.0,0.0,2.0,0.0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,3,0,1318.0,0.000,657126.000,568408.500,0.000,0.0,-163,4261.500,-373,-62,2.0,0.0,1.0,0.0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,7.000000,3.0,13.0,0.735043,0.538462,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.128205,0.000000,0.384615,0.136752,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,13,0,-14788.0,30735.000,3693858.660,163071.000,0.000,0.0,-13839,0.000,-2713,-483,2.0,0.0,11.0,0.0,1,0,0,9,3,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456250,3,0,3865.0,0.000,3086459.550,2232040.095,58268.385,0.0,-181,463703.895,-1002,-760,2.0,0.0,1.0,0.0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,29.000000,26.0,33.0,0.130259,0.071429,0.242424,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.252525,0.000000,0.757576,0.617216,0.000000,0.928571
456253,4,0,1122.0,0.000,3960000.000,1795833.000,0.000,0.0,-1013,175108.500,-919,-713,2.0,0.0,2.0,0.0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,29.250000,24.0,31.0,0.404906,0.387097,0.458333,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.459677,0.000000,0.612903,0.135417,0.000000,0.541667


#### Application Table

In [64]:
app_test = pd.read_csv(path+"application_test.csv")
app_train = pd.read_csv(path+"application_train.csv")

app = app_train.append(app_test).reset_index(drop=True)
app.set_index('SK_ID_CURR', inplace=True)

In [65]:
app['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

# APPLICATION credit_annuity_ratio: AMT_CREDIT / AMT_ANNUITY

app["CREDIT_ANNIUTY_RATIO"] = app['AMT_CREDIT'] / app['AMT_ANNUITY']

#Application credit_goods_price_ratio: AMT_CREDIT / AMT_GOODS_PRICE

app['CREDIT_GOODS_PRICE_RATIO'] = app['AMT_CREDIT'] / app['AMT_GOODS_PRICE']

#APPLICATION credit_downpayment: AMT_GOODS_PRICE - AMT_CREDIT

app['CREDIT_DOWNPAYMENT'] = app['AMT_GOODS_PRICE'] - app['AMT_CREDIT']

#APPLICATION AGE_INT: int(DAYS_BIRTH / -365)

app['AGE_INT'] = (app['DAYS_BIRTH'] / -365).astype(np.int64)


#Additional Ones From Laura

app['currEmp_to_age'] = app['DAYS_EMPLOYED'] / app['DAYS_BIRTH']
app['annuity_to_income'] = app['AMT_ANNUITY'] / (app['AMT_INCOME_TOTAL'] + 1)


app['NEW_CAR_TO_BIRTH_RATIO'] = np.where(app['DAYS_BIRTH'] != 0,
                                         app['OWN_CAR_AGE'] / app['DAYS_BIRTH'],
                                         np.nan)
app['NEW_CAR_TO_EMPLOY_RATIO'] = np.where(app['DAYS_EMPLOYED'] != 0,
                                          app['OWN_CAR_AGE'] / app['DAYS_EMPLOYED'],
                                          np.nan)
app['NEW_PHONE_TO_BIRTH_RATIO'] = np.where(app['DAYS_BIRTH'] != 0,
                                           app['DAYS_LAST_PHONE_CHANGE'] / app['DAYS_BIRTH'],
                                           np.nan)
app['NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER'] = np.where(app['DAYS_EMPLOYED'] != 0,
                                                    app['DAYS_LAST_PHONE_CHANGE'] \
                                                    / app['DAYS_EMPLOYED'],
                                                    np.nan)
app['NEW_CREDIT_TO_INCOME_RATIO'] = np.where(app['AMT_INCOME_TOTAL'] != 0,
                                             app['AMT_CREDIT'] / app['AMT_INCOME_TOTAL'],
                                             np.nan)

#Laura Code External Source Col's

app['ext_mean'] = app[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
app['ext_stdev'] = app[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
app['ext_product'] = app['EXT_SOURCE_1'] * app['EXT_SOURCE_2'] * app['EXT_SOURCE_3']


In [66]:
app['CODE_GENDER'].replace({'F':1,'M':0,'XNA':np.nan}, inplace=True)

In [67]:
app['FLAG_OWN_CAR'].replace({'Y':1, 'N':0}, inplace=True)
app['FLAG_OWN_REALTY'].replace({'Y':1, 'N':0}, inplace=True)

In [68]:
app['NAME_EDUCATION_TYPE'].replace({'Lower secondary':0, 'Secondary / secondary special':1,
                                    'Incomplete higher':2, 'Higher education':3,
                                    'Academic degree':4}, inplace=True)

In [69]:
app['EMERGENCYSTATE_MODE'].replace({'Yes':1, 'No':0}, inplace=True)

In [70]:
app = encoder(app)

In [71]:
app

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_EDUCATION_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,...,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0.0,0,1,0,202500.0,406597.5,24700.5,351000.0,1,0.018801,-9461,-637.0,-3648.0,-2120,NaN,1,1,0,1,1,0,1.0,2,2,10,0,0,0,0,0,0,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0
100003,0.0,1.0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,3,0.003541,-16765,-1188.0,-1186.0,-291,NaN,1,1,0,1,1,0,2.0,1,1,11,0,0,0,0,0,0,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0
100004,0.0,0.0,1,1,0,67500.0,135000.0,6750.0,135000.0,1,0.010032,-19046,-225.0,-4260.0,-2531,26.0,1,1,1,1,1,0,1.0,2,2,9,0,0,0,0,0,0,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100006,0.0,1.0,0,1,0,135000.0,312682.5,29686.5,297000.0,1,0.008019,-19005,-3039.0,-9833.0,-2437,NaN,1,1,0,1,0,0,2.0,2,2,17,0,0,0,0,0,0,NaN,0.650442,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100007,0.0,0.0,0,1,0,121500.0,513000.0,21865.5,513000.0,1,0.028663,-19932,-3038.0,-4311.0,-3458,NaN,1,1,0,1,0,0,1.0,2,2,11,0,0,0,0,1,1,NaN,0.322738,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456221,NaN,1.0,0,1,0,121500.0,412560.0,17473.5,270000.0,1,0.002042,-19970,-5169.0,-9094.0,-3399,NaN,1,1,1,1,1,0,1.0,3,3,16,0,0,0,0,0,0,NaN,0.648575,0.643026,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
456222,NaN,1.0,0,0,2,157500.0,622413.0,31909.5,495000.0,1,0.035792,-11186,-1149.0,-3015.0,-3003,N

In [72]:
all_data = app.join([prev_app_agg, pos_agg, insta_agg, credit_agg, bur_final])

In [73]:
del prev_app_agg, pos_agg, insta_agg, credit_agg, bur_final

In [74]:
test = all_data[all_data.index.isin(app_test['SK_ID_CURR'])].copy()

In [75]:
train = all_data[all_data.index.isin(app_train['SK_ID_CURR'])].copy()

In [76]:
# del app, app_train, app_test, all_data

In [77]:
test.drop('TARGET', axis=1, inplace=True)
train_labels = train.pop('TARGET')

In [78]:
train.shape

(307511, 1057)

In [79]:
train_d = train.dropna(axis=1, how='all')

In [80]:
columns = train_d.columns

In [81]:
columns

Index(['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'NAME_EDUCATION_TYPE', 'REGION_POPULATION_RELATIVE',
       ...
       'STATUS_4_maxbur_bal', 'STATUS_5_meanbur_bal', 'STATUS_5_minbur_bal',
       'STATUS_5_maxbur_bal', 'STATUS_C_meanbur_bal', 'STATUS_C_minbur_bal',
       'STATUS_C_maxbur_bal', 'STATUS_X_meanbur_bal', 'STATUS_X_minbur_bal',
       'STATUS_X_maxbur_bal'],
      dtype='object', length=1049)

In [82]:
from sklearn.impute import SimpleImputer
si = SimpleImputer()
train_i = si.fit_transform(train)
test_i = si.transform(test)

In [83]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_n = scaler.fit_transform(train_i)
test_n = scaler.transform(test_i)

In [84]:
train_feat = pd.DataFrame(train_n)
train_lab = pd.DataFrame(train_labels).reset_index()['TARGET']
test_feat = pd.DataFrame(test_n)

In [85]:
train_feat.columns = columns
test_feat.columns = columns

In [86]:
train_feat.to_pickle(path + "train_feat3.pkl")
train_lab.to_pickle(path + "train_lab3.pkl")
test_feat.to_pickle(path + "test_feat3.pkl")